In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
!pip install qq-training-wheels auquan_toolbox --upgrade

Requirement already up-to-date: auquan_toolbox in c:\users\nisarg\appdata\roaming\python\python38\site-packages (2.1.92)


In [9]:
from qq_training_wheels.momentum_trading import MomentumTradingParams
from backtester.trading_system import TradingSystem
from backtester.features.feature import Feature
import numpy as np

The class below implements all the logic you need to run the momentum backtester. Go through it and make sure you understand each part. You can run it first and make changes later to see if you made any improvements over the naive strategy.

There are 6 functions within the class:

- \_\_init\_\_
- getSymbolsToTrade
- getInstrumentFeatureConfigDicts
- getPredictions
- hurst_f
- updateCount

**__init__**

Initializes the class

**getSymbolsToTrade**

This is where we can select which stocks we want to test our strategy on. Here we're using just AAPL is it is the only ticker returned

**getInstrumentConfigDicts** 

This is the way that the toolbox creates features that we want to use in our logic. It's really important for resource optimisation at scale but can look a little daunting at first. We've created the features you'll need for you. If you're interested in learning more you can here: https://blog.quant-quest.com/toolbox-breakdown-getfeatureconfigdicts-function/

**getPrediction**

This again is fairly straight forward. We've included a few notes here, but for more detail: https://blog.quant-quest.com/toolbox-breakdown-getprediction-function/

Once you've calculated the hurst exponent, this should contain the logic to use it and make profitable trades.

**hurst_f**

This is your time to shine! This is where you will need to implement the hurst exponent as shown in the previous lecture. There are several different ways of calculating the hurst exponent, so we recommend you use the method shown in the lecture to allow other people to easily help you - if needed!

**updateCount**
A counter

In [10]:
class MyTradingFunctions():

    def __init__(self):
        self.count = 0
        # When to start trading
        self.start_date = '2015/01/02'
        # When to end trading
        self.end_date = '2017/08/31'
        self.params = {}

    def getSymbolsToTrade(self):
        '''
        Specify the stock names that you want to trade.
        '''
        return ['AAPL']

    def getInstrumentFeatureConfigDicts(self):
        '''
        Specify all Features you want to use by creating config dictionaries.
        Create one dictionary per feature and return them in an array.

        Feature config Dictionary have the following keys:

        featureId: a str for the type of feature you want to use
        featureKey: {optional} a str for the key you will use to call this feature
                    If not present, will just use featureId
        params: {optional} A dictionary with which contains other optional params if needed by the feature

        msDict = {
            'featureKey': 'ms_5',
            'featureId': 'moving_sum',
            'params': {
                'period': 5,
                'featureName': 'basis'
            }
        }

        return [msDict]

        You can now use this feature by in getPRediction() calling it's featureKey, 'ms_5'
        '''

        ma1Dict = {
            'featureKey': 'ma_90',
            'featureId': 'moving_average',
            'params': {
                'period': 90,
                'featureName': 'adjClose'
            }
        }
        mom30Dict = {
            'featureKey': 'mom_30',
            'featureId': 'momentum',
            'params': {
                'period': 30,
                'featureName': 'adjClose'
            }
        }
        mom10Dict = {
            'featureKey': 'mom_10',
            'featureId': 'momentum',
            'params': {
                'period': 10,
                'featureName': 'adjClose'
            }
        }
        
        return [ma1Dict, mom10Dict, mom30Dict]

    def getPrediction(self, time, updateNum, instrumentManager, predictions):
        '''
        Combine all the features to create the desired predictions for each stock.
        'predictions' is Pandas Series with stock as index and predictions as values
        We first call the holder for all the instrument features for all stocks as
            lookbackInstrumentFeatures = instrumentManager.getLookbackInstrumentFeatures()
        Then call the dataframe for a feature using its feature_key as
            ms5Data = lookbackInstrumentFeatures.getFeatureDf('ms_5')
        This returns a dataFrame for that feature for ALL stocks for all times upto lookback time
        Now you can call just the last data point for ALL stocks as
            ms5 = ms5Data.iloc[-1]
        You can call last datapoint for one stock 'ABC' as
            value_for_abs = ms5['ABC']

        Output of the prediction function is used by the toolbox to make further trading decisions and evaluate your score.
        '''

        self.updateCount() # uncomment if you want a counter

        # holder for all the instrument features for all instruments
        lookbackInstrumentFeatures = instrumentManager.getLookbackInstrumentFeatures()
        
        def hurst_f(input_ts, lags_to_test=20):  
            # interpretation of return value
            # hurst < 0.5 - input_ts is mean reverting
            # hurst = 0.5 - input_ts is effectively random/geometric brownian motion
            # hurst > 0.5 - input_ts is trending
            tau = []
            lagvec = []  
            #  Step through the different lags  
            for lag in range(2, lags_to_test):  
                #  produce price difference with lag  
                pp = np.subtract(input_ts[lag:].values, input_ts[:-lag].values)  
                #  Write the different lags into a vector  
                lagvec.append(lag)  
                #  Calculate the variance of the differnce vector  
                tau.append(np.sqrt(np.std(pp)))  
            #  linear fit to double-log graph (gives power)  
            m = np.polyfit(np.log10(lagvec), np.log10(tau), 1)  
            # calculate hurst  
            hurst = m[0]*2
            print(hurst)
            return hurst  

        # dataframe for a historical instrument feature (ma_90 in this case). The index is the timestamps
        # of upto lookback data points. The columns of this dataframe are the stock symbols/instrumentIds.
        mom10Data = lookbackInstrumentFeatures.getFeatureDf('mom_10')
        mom30Data = lookbackInstrumentFeatures.getFeatureDf('mom_30')
        ma90Data = lookbackInstrumentFeatures.getFeatureDf('ma_90')
        
        # Here we are making predictions on the basis of Hurst exponent if enough data is available, otherwise
        # we simply get out of our position
        if len(ma90Data.index)>20:
            mom30 = mom30Data.iloc[-1]
            mom10 = mom10Data.iloc[-1]
            ma90 = ma90Data.iloc[-1]
            
            # Calculate Hurst Exponent
            hurst = ma90Data.apply(hurst_f, axis=0)
            # Go long if Hurst > 0.5 and both long term and short term momentum are positive
            predictions[(hurst > 0.5) & (mom30 > 0) & (mom10 > 0)] = 1 
            # Go short if Hurst > 0.5 and both long term and short term momentum are negative
            predictions[(hurst > 0.5) & (mom30 <= 0) & (mom10 <= 0)] = 0 
            
            # Get out of position if Hurst > 0.5 and long term momentum is positive while short term is negative
            predictions[(hurst > 0.5) & (mom30 > 0) & (mom10 <= 0)] = 0.5
            # Get out of position if Hurst > 0.5 and long term momentum is negative while short term is positive
            predictions[(hurst > 0.5) & (mom30 <= 0) & (mom10 > 0)] = 0.5
            
            # Get out of position if Hurst < 0.5
            predictions[hurst <= 0.5] = 0.5        
        else:
            # If no sufficient data then don't take any positions
            predictions.values[:] = 0.5
        return predictions

    def updateCount(self):
        self.count = self.count + 1

In [11]:
tf = MyTradingFunctions()
tsParams = MomentumTradingParams(tf)
tradingSystem = TradingSystem(tsParams)

Processing data for stock: AAPL
20% done...
40% done...
60% done...
80% done...
Logging all the available market metrics in tensorboard
Logging all the available instrument metrics in tensorboard


In [12]:
results = tradingSystem.startTrading()

2015-01-02 00:00:00
2015-01-05 00:00:00
AAPL
pnl: 0.00
2015-01-06 00:00:00
AAPL
pnl: 0.00
2015-01-07 00:00:00
AAPL
pnl: 0.00
2015-01-08 00:00:00
AAPL
pnl: 0.00
2015-01-09 00:00:00
AAPL
pnl: 0.00
2015-01-12 00:00:00
AAPL
pnl: 0.00
2015-01-13 00:00:00
AAPL
pnl: 0.00
2015-01-14 00:00:00
AAPL
pnl: 0.00
2015-01-15 00:00:00
AAPL
pnl: 0.00
2015-01-16 00:00:00
AAPL
pnl: 0.00
2015-01-19 00:00:00
AAPL
pnl: 0.00
2015-01-20 00:00:00
AAPL
pnl: 0.00
2015-01-21 00:00:00
AAPL
pnl: 0.00
2015-01-22 00:00:00
AAPL
pnl: 0.00
2015-01-23 00:00:00
AAPL
pnl: 0.00
2015-01-26 00:00:00
AAPL
pnl: 0.00
2015-01-27 00:00:00
AAPL
pnl: 0.00
2015-01-28 00:00:00
AAPL
pnl: 0.00
2015-01-29 00:00:00
AAPL
pnl: 0.00
2015-01-30 00:00:00
0.09157947382846832
AAPL
pnl: 0.00
2015-02-02 00:00:00
0.09615896685745046
AAPL
pnl: 0.00
2015-02-03 00:00:00
0.08435527376771398
AAPL
pnl: 0.00
2015-02-04 00:00:00
0.07706926228238557
AAPL
pnl: 0.00
2015-02-05 00:00:00
0.08728331598306384
AAPL
pnl: 0.00
2015-02-06 00:00:00
0.11144222489905295


AAPL
pnl: -5.34
2015-07-28 00:00:00
0.8155125905173196
AAPL
pnl: -5.34
Position changed to: -1.00
2015-07-29 00:00:00
0.8125916439076009
AAPL
pnl: -5.48
2015-07-30 00:00:00
0.8110545686733693
AAPL
pnl: -5.39
2015-07-31 00:00:00
0.8112274656280053
AAPL
pnl: -5.25
2015-08-03 00:00:00
0.8132939708793078
AAPL
pnl: -5.01
2015-08-04 00:00:00
0.8165437907857737
AAPL
pnl: -4.36
2015-08-05 00:00:00
0.822278809854348
AAPL
pnl: -3.49
2015-08-06 00:00:00
0.8327750325370603
AAPL
pnl: -3.66
2015-08-07 00:00:00
0.845856123267921
AAPL
pnl: -3.72
2015-08-10 00:00:00
0.8560074021190803
AAPL
pnl: -3.81
2015-08-11 00:00:00
0.8636472027303356
AAPL
pnl: -4.77
2015-08-12 00:00:00
0.8712409019907636
AAPL
pnl: -3.35
2015-08-13 00:00:00
0.8779231091942117
AAPL
pnl: -3.75
2015-08-14 00:00:00
0.8848985389682522
AAPL
pnl: -3.73
2015-08-17 00:00:00
0.8915085156209899
AAPL
pnl: -3.91
2015-08-18 00:00:00
0.8969785061907276
AAPL
pnl: -4.19
2015-08-19 00:00:00
0.9013104253762356
AAPL
pnl: -4.04
Position changed to: 0.0

2016-02-03 00:00:00
0.8827552861140932
AAPL
pnl: 1.22
Position changed to: -1.00
2016-02-04 00:00:00
0.8869816634500317
AAPL
pnl: 0.79
2016-02-05 00:00:00
0.8928144902164392
AAPL
pnl: 0.61
Position changed to: 0.00
2016-02-08 00:00:00
0.8967516149339786
AAPL
pnl: 0.61
Position changed to: -1.00
2016-02-09 00:00:00
0.8993548344175967
AAPL
pnl: 0.38
2016-02-10 00:00:00
0.9019182713212714
AAPL
pnl: 0.39
2016-02-11 00:00:00
0.9053819364736944
AAPL
pnl: 0.56
Position changed to: 0.00
2016-02-12 00:00:00
0.9095337641942502
AAPL
pnl: 0.56
2016-02-15 00:00:00
0.9121716319687009
AAPL
pnl: 0.56
Position changed to: -1.00
2016-02-16 00:00:00
0.913625233949768
AAPL
pnl: 0.56
2016-02-17 00:00:00
0.9160347767144814
AAPL
pnl: -0.06
Position changed to: 0.00
2016-02-18 00:00:00
0.9188674150571975
AAPL
pnl: -0.06
2016-02-19 00:00:00
0.9198803715489535
AAPL
pnl: -0.06
Position changed to: 1.00
2016-02-22 00:00:00
0.9198653755012483
AAPL
pnl: -0.11
Position changed to: 0.00
2016-02-23 00:00:00
0.91865078

2016-08-24 00:00:00
0.8580704077990687
AAPL
pnl: 4.21
Position changed to: 0.00
2016-08-25 00:00:00
0.8628183750949984
AAPL
pnl: 4.21
Position changed to: 1.00
2016-08-26 00:00:00
0.8630180036078997
AAPL
pnl: 4.10
2016-08-29 00:00:00
0.8593412598339
AAPL
pnl: 3.96
Position changed to: 0.00
2016-08-30 00:00:00
0.8547614460440578
AAPL
pnl: 3.96
2016-08-31 00:00:00
0.846828811844236
AAPL
pnl: 3.96
2016-09-01 00:00:00
0.8330111023840803
AAPL
pnl: 3.96
2016-09-02 00:00:00
0.8150733156043376
AAPL
pnl: 3.96
2016-09-05 00:00:00
0.7972827810861998
AAPL
pnl: 3.96
2016-09-06 00:00:00
0.7852525294992115
AAPL
pnl: 3.96
2016-09-07 00:00:00
0.7773275937900328
AAPL
pnl: 3.96
2016-09-08 00:00:00
0.7738803385483229
AAPL
pnl: 3.96
2016-09-09 00:00:00
0.7754720807811422
AAPL
pnl: 3.96
Position changed to: 1.00
2016-09-12 00:00:00
0.7796294255789713
AAPL
pnl: 3.40
Position changed to: 0.00
2016-09-13 00:00:00
0.7845937242961469
AAPL
pnl: 3.40
Position changed to: -1.00
2016-09-14 00:00:00
0.788080126079758

0.8709913851218719
AAPL
pnl: 8.10
2017-03-10 00:00:00
0.8663876510034102
AAPL
pnl: 8.03
2017-03-13 00:00:00
0.8634827142689322
AAPL
pnl: 8.14
2017-03-14 00:00:00
0.8633738283802336
AAPL
pnl: 8.15
2017-03-15 00:00:00
0.8646943867999758
AAPL
pnl: 8.10
2017-03-16 00:00:00
0.8649359527947359
AAPL
pnl: 8.45
Position changed to: 0.00
2017-03-17 00:00:00
0.8651484072137997
AAPL
pnl: 8.45
Position changed to: 1.00
2017-03-20 00:00:00
0.8642561162313007
AAPL
pnl: 8.28
2017-03-21 00:00:00
0.8640749587150685
AAPL
pnl: 8.63
2017-03-22 00:00:00
0.8664731409652705
AAPL
pnl: 8.25
2017-03-23 00:00:00
0.8709460968165755
AAPL
pnl: 8.62
2017-03-24 00:00:00
0.8768648371305776
AAPL
pnl: 8.50
2017-03-27 00:00:00
0.8840761663986555
AAPL
pnl: 8.43
2017-03-28 00:00:00
0.8911612683755794
AAPL
pnl: 8.49
2017-03-29 00:00:00
0.8973355649881495
AAPL
pnl: 9.18
2017-03-30 00:00:00
0.9031098739250973
AAPL
pnl: 9.26
2017-03-31 00:00:00
0.9090195565112124
AAPL
pnl: 9.21
2017-04-03 00:00:00
0.914936573811759
AAPL
pnl: 9.